In [1]:
from mdagent import MDAgent

In [2]:
#todays date and time
import datetime

start = datetime.datetime.now()
date = start.strftime("%Y-%m-%d")
print("date:",date)
time = start.strftime("%H:%M:%S")
print("time:",time)

date: 2024-08-06
time: 10:46:22


In [3]:
model="llama-v3p1-405b-instruct"
mda = MDAgent(
    agent_type='Structured', 
    model=model, 
    tools_model=model,
    use_memory=False, 
    top_k_tools="all"
)
print("LLM: ", mda.llm.model_name,"\nTemperature: ", mda.llm.temperature)

LLM:  accounts/fireworks/models/llama-v3p1-405b-instruct 
Temperature:  0.1


In [4]:
prompt = '''Simulate hemoglobin oxygenated (1A3N) and de-oxygenated (6BB5).'''
answer = mda.run(prompt)



> Entering new AgentExecutor chain...
Thought: To simulate hemoglobin in both its oxygenated and de-oxygenated states, I need to set up and run a molecular dynamics simulation for each state. The first step is to download the PDB files for oxygenated (1A3N) and de-oxygenated (6BB5) hemoglobin.

Action: PDBFileDownloader
Action Input: 
{
  "query": "1A3N"
}<|eom_id|><|start_header_id|>assistant<|end_header_id|>

Thought: To simulate hemoglobin in both its oxygenated and de-oxygenated states, I need to set up and run a molecular dynamics simulation for each state. The first step is to download the PDB files for oxygenated (1A3N) and de-oxygenated (6BB5) hemoglobin.

Action: PDBFileDownloader
Action Input: 
{
  "query": "1A3N"
}<|eom_id|><|start_header_id|>assistant<|end_header_id|>



> Finished chain.


In [5]:
end = datetime.datetime.now()
elapsed = end - start
print(f"duration: {elapsed.total_seconds()/60:.2f} minutes")

duration: 0.09 minutes


# Confirm all files are downloaded or made

In [6]:
registry = mda.path_registry
all_names = registry.list_path_names()
print(all_names)

No names found. The JSON file is empty or does not contain name mappings.


In [7]:
# # ensure all files exist
# file_ids = [....]
# assert all(f in all_names for f in file_ids), "Not all file ids are present in path registry"

# for f in file_ids:
#     match = re.search(rf"{f}_\d+", all_names)
#     if match is None: 
#         print(f"this file id {f} is not in path registry")
#     file_id = match.group(0)
#     path = registry.get_mapped_path(file_id)
#     if not os.path.exists(path):
#         print(f"the file for {file_id} doesn't exist")

In [8]:
# ensure input to each simulation step is 13AN and 6BB5 -- we can verify via file id? top file, maybe? 

# Experiment Result:

1. Completed without Exception or TimeOut Errors ✅
2. Attempted all necessary steps ❌
3. No hallucination ❌
4. Logic make sense ❌
5. Correct Answer ❌
6. Number of steps completed: 0️⃣
7. % steps completed: 0️⃣%